<a href="https://colab.research.google.com/github/anshudaur/TensorFlowProjects/blob/master/LanguageModelRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras
import time
import pandas as pd
import copy
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

In [2]:
!python3 prepare_data.py shakespeare_input.txt skp 

2020-05-26 05:08:15.168346: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Split input into 22981 sequences...
Serialized 100 sequences...
Serialized 200 sequences...
Serialized 300 sequences...
Serialized 400 sequences...
Serialized 500 sequences...
Serialized 600 sequences...
Serialized 700 sequences...
Serialized 800 sequences...
Serialized 900 sequences...
Serialized 1000 sequences...
Serialized 1100 sequences...
Serialized 1200 sequences...
Serialized 1300 sequences...
Serialized 1400 sequences...
Serialized 1500 sequences...
Serialized 1600 sequences...
Serialized 1700 sequences...
Serialized 1800 sequences...
Serialized 1900 sequences...
Serialized 2000 sequences...
Serialized 2100 sequences...
Serialized 2200 sequences...
Serialized 2300 sequences...
Serialized 2400 sequences...
Serialized 2500 sequences...
Serialized 2600 sequences...
Serialized 2700 sequences...
Serialized 2800 sequences...
Serialized 2900

In [3]:
from prepare_data import parse_seq
import pickle
# this is just a datasets of "bytes" (not understandable)
data = tf.data.TFRecordDataset("skp.tfrecords")

# this maps a parser function that properly interprets the bytes over the dataset
# (with fixed sequence length 200)
# if you change the sequence length in preprocessing you also need to change it here
data = data.map(lambda x: parse_seq(x, 200))

# a map from characters to indices
vocab = pickle.load(open("skp_vocab", mode="rb"))
vocab_size = len(vocab)
# inverse mapping: indices to characters
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}

print(vocab)
print(vocab_size)

{'g': 1, 'P': 2, 'Y': 3, 'O': 4, '[': 5, 'A': 6, "'": 7, 'b': 8, 'D': 9, 'N': 10, '!': 11, '-': 12, 'q': 13, 'i': 14, 'I': 15, 'L': 16, 'y': 17, 'V': 18, '3': 19, 'p': 20, 'J': 21, 'o': 22, '&': 23, 'u': 24, 'W': 25, 'F': 26, 'M': 27, 'c': 28, 'k': 29, 'z': 30, 'x': 31, 'f': 32, 'B': 33, 'r': 34, 'U': 35, 'X': 36, '$': 37, 'T': 38, '.': 39, 'w': 40, 'd': 41, 'H': 42, 'G': 43, 'C': 44, ':': 45, '?': 46, 'v': 47, 'R': 48, '\n': 49, 'e': 50, 'h': 51, 'n': 52, 'm': 53, ';': 54, 'j': 55, 's': 56, 'Q': 57, ' ': 58, 'Z': 59, 'E': 60, 'K': 61, 'a': 62, ']': 63, 't': 64, 'S': 65, ',': 66, 'l': 67, '<S>': 0}
68


In [0]:
def one_hot_encode(data):
  #function to convert data to one hot encoding format for each character 
  data = tf.one_hot(indices=data,depth=vocab_size)
  return data;

In [9]:
sequence_length = 200 #already given
#each batch should be batch_size x seq_len x vocab_size
batch_size=128

test_data=data
test_data = test_data.batch(batch_size,drop_remainder=True)
print("Test data : ", test_data)

train_data = data.map(one_hot_encode)
train_data = train_data.batch(batch_size,drop_remainder=True)
print("Train data : ",train_data)


optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy()

Test data :  <BatchDataset shapes: (128, 200), types: tf.int32>
Train data :  <BatchDataset shapes: (128, 200, 68), types: tf.float32>


 input is turned from batch_size x seq_len into batch_size x seq_len x vocab_size.

In [0]:
#@tf.function
def model(train_data,test_data,time_steps,hidden_units,batch_size,epochs):  
  n_h = hidden_units  
  #input layer variables
  W_xh = tf.Variable(tf.initializers.GlorotUniform(seed = 1)([vocab_size,n_h])) #shape=(68,512) 
  #hidden layer variables
  W_hh = tf.Variable(tf.initializers.GlorotUniform(seed = 1)([n_h, n_h])) #shape=(512, 512) 
  b_h = tf.Variable(tf.zeros([n_h]))
  #output layer variables
  W_ho = tf.Variable(tf.initializers.GlorotUniform(seed = 1)([n_h,vocab_size])) #shape=(512,68) 
  b_o = tf.Variable(tf.zeros([vocab_size]))
  model_loss = 0
  for epoch in range(epochs):    
    for batch_num,(x, y) in enumerate(zip(train_data,test_data)):
      with tf.GradientTape() as tape: 
        recurrent_state =  tf.Variable(tf.initializers.GlorotUniform(seed=1)([batch_size, n_h]))
        xent = tf.TensorArray(tf.float32, size=200 , dynamic_size= True) #change
        total_loss = 0.0  
        for time_step in range(time_steps-1):          
          x_seq = x[:,time_step] #shape=(128, each character (1,2,3,...)) 
          y_seq = y[:, time_step+1] #next char sequence , shape=(128) of single characters
          #print(x_seq.shape) #(128,68)
          #print(y_seq.shape) #(128,)
          recurrent_state = tf.nn.tanh( tf.matmul(x_seq, W_xh) + tf.matmul(recurrent_state, W_hh) + b_h)
          logits = tf.matmul(recurrent_state, W_ho) + b_o #change- no need of activation
          loss = loss_fn(y[:, time_step+1], logits) #change
          xent = xent.write(time_step, loss) #change
        # softmax loss between the output and the target at this time step. 
        # Keep in mind that the targets are just the input shifted by one time step 
        xent = xent.stack() #change            
        total_loss = tf.reduce_mean(xent) #change
      if not batch_num % 100:  
        print("loss :: ",total_loss)    
      grads = tape.gradient(total_loss,[W_xh, W_hh, b_h, W_ho, b_o])
      optimizer.apply_gradients(zip(grads, [W_xh, W_hh, b_h, W_ho, b_o]))
      metric(y[:, time_step+1], logits) 
         
    # Display metrics at the end of each epoch.
    accuracy = metric.result()    
    print('Training acc over epoch' , epoch,' : ',(float(accuracy)))
    # Reset training metrics at the end of each epoch
    metric.reset_states()

57% training accuracy , Loss=1.5 for 30 epochs

In [11]:
# model variables : recommended 512 hidden units (batch size 128 and Adam optimizer, 20 or so )
model(train_data,test_data,time_steps=200,hidden_units=512,batch_size=128,epochs=30)

loss ::  tf.Tensor(4.2326527, shape=(), dtype=float32)
loss ::  tf.Tensor(2.6630497, shape=(), dtype=float32)
Training acc over epoch 0  :  0.25964561104774475
loss ::  tf.Tensor(2.3674576, shape=(), dtype=float32)
loss ::  tf.Tensor(2.2905195, shape=(), dtype=float32)
Training acc over epoch 1  :  0.3634340167045593
loss ::  tf.Tensor(2.198036, shape=(), dtype=float32)
loss ::  tf.Tensor(2.1967678, shape=(), dtype=float32)
Training acc over epoch 2  :  0.3858240246772766
loss ::  tf.Tensor(2.117271, shape=(), dtype=float32)
loss ::  tf.Tensor(2.130148, shape=(), dtype=float32)
Training acc over epoch 3  :  0.40245285630226135
loss ::  tf.Tensor(2.0464087, shape=(), dtype=float32)
loss ::  tf.Tensor(2.0652843, shape=(), dtype=float32)
Training acc over epoch 4  :  0.41833972930908203
loss ::  tf.Tensor(1.9855852, shape=(), dtype=float32)
loss ::  tf.Tensor(1.9972818, shape=(), dtype=float32)
Training acc over epoch 5  :  0.4366270899772644
loss ::  tf.Tensor(1.9272236, shape=(), dtype=